### Importing libs

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import hvplot.pandas
import holoviews as hv

## Second df
#### This data contains data on City of Hartford real estate sales for the last two years, with comprehensive records including property ID, parcel ID, sale date, sale price and more. 

In [ ]:
df2_path = Path('csv_resources/real-estate-sales-730-days-1.csv')
df2 = pd.read_csv(df2_path,index_col='SaleDate', infer_datetime_format=True, parse_dates=True)
df2.sort_index(inplace=True)
df2= df2.replace(np.nan, 'Unknown')
display(df2.head(10))



df2_cleaned = df2 [['OwnerFirstName','OwnerLastName','StreetNameAndWay','PrimaryGrantor', 'SalePrice', 'LegalReference','ParcelID']]

df2_final = df2_cleaned.drop(columns=['OwnerFirstName','OwnerLastName','StreetNameAndWay','PrimaryGrantor','SalePrice','LegalReference'])
df2_final
display(df2_final.head(10))
display(df2_final.tail(10))
df2_final.columns

### Third df
#### This table contains property sales information including sale date, price, and amounts for properties within Fairfax County

In [ ]:
df3_path = Path('csv_resources/Tax_Administration_s_Real_Estate_-_Sales_Data_cutdown1.csv')
#df3 = pd.read_csv(df3_path)
df3 = pd.read_csv(df3_path, index_col='SALEDT', infer_datetime_format=True, parse_dates=True)
df3.sort_index(inplace=True)
df3 = df3.replace(np.nan, 'Uknown')
display(df3)

df3.columns

In [ ]:
df3_cleaned = df3.drop(columns=['Unnamed: 0'])



In [ ]:
df3_cleaned.index.names = ['SaleDate']
df3_combine = df3_cleaned.query('index > "2014-12-31"')
display(df3_combine.head(10))
display(df3_combine.tail(10))
df3_combine.dtypes


### Combined DF 2 & DF 3

In [ ]:
joined_df_friction = pd.merge(df2_final,
                               df3_combine, 
                               left_index = True, 
                               right_index = True,
                               how = "outer"
)
display(joined_df_friction)

### Last cleaning

In [ ]:
sale_date_df = joined_df_friction.drop(columns=['ParcelID'])
display(sale_date_df)


### Sale date and Estimated move in date

In [ ]:
sale_date_df['Estimated Move in Date'] = sale_date_df.index + pd.to_timedelta(np.random.randint(25, 70, size=len(sale_date_df)), unit='D')
display(sale_date_df)

### Data + Days-difference and indexed

In [ ]:
sale_date_df['days_difference'] = (sale_date_df['Estimated Move in Date'] - sale_date_df.index).dt.days
display(sale_date_df)

## Grouped by year

In [ ]:
sale_date_df['year'] = sale_date_df.index.year
yearly_average = sale_date_df.groupby('year')['days_difference'].mean().reset_index()
display(yearly_average)

## Created Dataframe with Sale and Move in dates

In [ ]:
housing_data = sale_date_df.reset_index()
housing_data = housing_data.drop(columns=['year'])
display(housing_data.tail(10))

### Monthly average

In [ ]:
housing_data['month'] = housing_data['SaleDate'].dt.month
housing_data['year'] = housing_data['SaleDate'].dt.year
result = housing_data.groupby(['year', 'month'])['days_difference'].mean()
result = result.reset_index()

display(result)

## Jan 2023 transactions

In [ ]:
start_date = '2020-01-01'
end_date = '2020-01-31'
mask = (housing_data['SaleDate'] >= start_date) & (housing_data['SaleDate'] <= end_date)
janurary_df = housing_data.loc[mask]
janurary_df = janurary_df.drop(columns=['month','year'])
jan_plot = janurary_df.iloc[::75]

display(jan_plot)


## Jan 2023 Bar House sales Plot

In [ ]:
jan_plot['SaleDate'] = jan_plot['SaleDate'].dt.date

In [ ]:

def get_color(value):
    if 25 <= value < 40:
        return '#8bab7c'
    elif 40 <= value < 50:
       return '#d8cfad'
    elif 50 <= value < 60:
        return '#d69163'
    elif 60 <=value < 70:
        return '#de425b'
    else:
        return 'gray'

jan_plot['color'] = jan_plot['days_difference'].apply(get_color)

jan_plot.hvplot.bar(
    x ='SaleDate',
    y ='days_difference',
    xlabel="Purchase Date",
    ylabel="Days Waiting to Move In",
    title="The Wait After The Purchase",
    rot = 80,
    width = 900,
    height = 500,
    color = 'color',
    ).opts(
    bgcolor='#003f5c',
    show_grid=True
    )
    

### Avergage move in days plot

In [ ]:

yearly_average.hvplot.line(
    x ='year',
    y ='days_difference',
    xlabel="Purchase year",
    ylabel="Days Waiting to Move In",
    title="Average waiting period to move in",
    rot = 45,
    width = 700,
    height = 400,
    ylim = (45,50),
    color = '#ffa600',
    line_width=3
  
    ) .opts (
    bgcolor='#003f5c',
    show_grid=True,
    gridstyle={'grid_line_color': 'grey'}
    )

### Dual Line plot, Sales Vs Move in date 

In [ ]:
jan_plot.hvplot.line(
    x ='index',
    y =['SaleDate', 'Estimated Move in Date'],
    xlabel="House Unit",
    ylabel="Dates",
    title="Purchase date VS Move-In dates",
    rot = 80,
    width = 900,
    height = 500,
    line_dash=['solid', 'dotted'],
    line_width=[3, 3],
    color=['#bc5090', '#ffa600']

).opts(
    xformatter = '%.0f',
    bgcolor='#003f5c',
    show_grid=True,
    gridstyle={'grid_line_color': 'grey'}
)

### Df for security

#### This Dataframe contains nationwide real estate records from 2017 

In [ ]:
sec_df1_path =Path('csv_resources/hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels_cutdown.csv') 
sec_df1 = pd.read_csv(sec_df1_path)
display(sec_df1)
sec_df1.columns
sec_info= sec_df1.drop(columns=['Unnamed: 0','agency_code','msamd_name','census_tract_number','co_applicant_ethnicity_name','co_applicant_sex_name','denial_reason_name_1','hoepa_status_name']) 
display(sec_info)


In [ ]:
sec_df2_path =Path('csv_resources/real-estate-sales-730-days-1.csv') 
sec_df2 = pd.read_csv(sec_df2_path, index_col='OwnerFirstName')
display(sec_df2)

In [ ]:
sec_df2.columns
sec_df_clean = sec_df2.drop(columns=['xrCompositeLandUseID', 'xrBuildingTypeID','xrSalesValidityID', 'xrDeedID',
                                     'xrPrimaryNeighborhoodID', 'LandSF',
                                    'TotalFinishedArea', 'LivingUnits','LocationStartNumber',
                                    ])
sec_df_clean.columns

sec_df_clean2 = sec_df_clean[['OwnerLastName','ParcelID','PropertyID', 'ApartmentUnitNumber','PrimaryGrantor',
       'SaleDate', 'SalePrice', 'TotalAppraisedValue', 'LegalReference']]
display(sec_df_clean2.head(20))